In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch
from torch.quantization import quantize_dynamic
from torchvision import datasets, transforms


In [9]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

X_train = mnist_trainset.data.numpy().reshape(len(mnist_trainset), -1)
y_train = mnist_trainset.targets.numpy()
X_test = mnist_testset.data.numpy().reshape(len(mnist_testset), -1)
y_test = mnist_testset.targets.numpy()


In [10]:
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [11]:
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Model Accuracy:", accuracy)


Logistic Regression Model Accuracy: 0.92


In [12]:
import joblib
import os

joblib.dump(logistic_model, "logistic_model.pkl")
model_size = os.path.getsize("logistic_model.pkl") / 1024
print("Logistic Regression Model Size:", model_size, "KB")


Logistic Regression Model Size: 62.2255859375 KB


In [13]:
import time

start_time = time.time()
logistic_model.predict(X_test[:100])
inference_time = (time.time() - start_time) / 100
print("Logistic Regression Model Inference Time (per sample):", inference_time, "seconds")


Logistic Regression Model Inference Time (per sample): 3.467798233032226e-05 seconds


In [14]:
def quantize_model(model, scale_factor=2**7):
    model.coef_ = (model.coef_ * scale_factor).astype(np.int8)
    model.intercept_ = (model.intercept_ * scale_factor).astype(np.int8)
    return model


In [15]:
def quantized_inference(model, X):
    X_quantized = (X / 2**7).astype(np.float32)
    return model.predict(X_quantized)


In [16]:
quantized_model = quantize_model(logistic_model)
y_quantized_pred = quantized_inference(quantized_model, X_test)
quantized_accuracy = accuracy_score(y_test, y_quantized_pred)
print("Quantized Model Accuracy:", quantized_accuracy)


Quantized Model Accuracy: 0.1962


In [17]:
joblib.dump(quantized_model, "quantized_model.pkl")
quantized_model_size = os.path.getsize("quantized_model.pkl") / 1024
print("Quantized Model Size:", quantized_model_size, "KB")


Quantized Model Size: 8.5791015625 KB


In [18]:
start_time = time.time()
quantized_inference(quantized_model, X_test[:100])
quantized_inference_time = (time.time() - start_time) / 100
print("Quantized Model Inference Time (per sample):", quantized_inference_time, "seconds")


Quantized Model Inference Time (per sample): 3.6795139312744144e-05 seconds


In [19]:
print("\nComparison Results:")
print("Original Model Size:", model_size, "KB")
print("Quantized Model Size:", quantized_model_size, "KB")
print("Original Inference Time:", inference_time, "seconds per sample")
print("Quantized Inference Time:", quantized_inference_time, "seconds per sample")
print("Original Model Accuracy:", accuracy)
print("Quantized Model Accuracy:", quantized_accuracy)



Comparison Results:
Original Model Size: 62.2255859375 KB
Quantized Model Size: 8.5791015625 KB
Original Inference Time: 3.467798233032226e-05 seconds per sample
Quantized Inference Time: 3.6795139312744144e-05 seconds per sample
Original Model Accuracy: 0.92
Quantized Model Accuracy: 0.1962
